<a href="https://colab.research.google.com/github/erodola/DLAI-s2-2023/blob/main/labs/04/4_Logistic_Regression_and_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning

# Tutorial 5: Principal Component Analysis

In this tutorial, we will cover:

- Principal components and dimensionality reduction
- Eigenvalues, SVD decomposition, and power iterations
- Learning parameter spaces

Authors:

- Prof. Emanuele Rodolà

Course:

- Lectures and notebooks at https://github.com/erodola/ML-s2-2024/

# Imports and utilities

In [ ]:
# @title import dependencies

import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn import decomposition
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from tqdm import tqdm

In [ ]:
# @title reproducibility stuff

import random
np.random.seed(42)
random.seed(0)

In [ ]:
# @title visualization utils

!pip install trimesh==4.2.4 rtree==1.2.0

import trimesh

def render(mesh, height_pix: int = 200):
  """ Renders a trimesh as a grayscale image.

  :Authors:
    Emanuele Rodolà
  """

  # center
  mesh.apply_translation(-mesh.centroid)

  # rotate
  R = trimesh.transformations.rotation_matrix(-np.pi/10, [0, 1, 0], point=None)
  mesh.apply_transform(R)

  # normalize
  mesh.apply_scale(1 / max(np.max(mesh.vertices, axis=0) - np.min(mesh.vertices, axis=0)))

  # rescale
  mesh.apply_scale(height_pix)

  # translate
  mesh.apply_translation(-np.min(mesh.vertices, axis=0))

  Vq = np.round(mesh.vertices).astype(int)
  w, h, _ = 1 + np.max(Vq, axis=0).astype(int)

  ray_origins = np.zeros((w*h, 3), dtype=np.float64)
  ray_origins[:, 0] = (np.arange(w)[:, None] * np.ones(h)).reshape(-1)
  ray_origins[:, 1] = (np.ones(w)[:, None] * np.arange(h)).reshape(-1)
  ray_origins[:, 2] = -10

  view_dir = np.array([0, 0, 1], dtype=np.float64)
  view_dir /= np.sqrt(np.sum(view_dir**2))

  ray_directions = np.tile(view_dir, (w*h, 1))

  locations, _, index_tri = mesh.ray.intersects_location(ray_origins=ray_origins, ray_directions=ray_directions)

  img = np.ones((w, h), dtype=np.float32)
  depths = np.empty((w, h), dtype=np.float32)

  light_dir = np.array([0, 0, 1], dtype=np.float32)
  light_dir /= np.sqrt(np.sum(light_dir**2))

  shades = np.dot(mesh.face_normals[index_tri], light_dir)

  for idx, p in enumerate(locations):
      j, i, z = int(p[0]), int(p[1]), p[2]
      if depths[j, i] is None or depths[j, i] < z:
          depths[j, i] = z
          img[j, i] = shades[idx]

  return img

# Principal components

Let's start simple!

We generate a bunch of 2D data points, and we'll use them in the first part of this notebook to play a bit.

In [ ]:
n = 50
xs = np.linspace(0, 1, n)
ys = 1.7 * xs + 2.1 + np.random.rand(n) * 1.1
X = np.stack((xs, ys), axis=0).T

plt.figure(figsize=(5,2))
plt.scatter(X[:,0], X[:,1], s=5, color='red', zorder=2)
plt.grid(True)
plt.show()

Take note of the shape of `X`; here we are storing each data point as a _row_ in the **data matrix $\mathbf{X}$**:

In [ ]:
X.shape

In theory class, we derived the $k$ principal components of the data as the top-$k$ eigenvectors of the **covariance matrix $\mathbf{C}=\mathbf{X}^\top \mathbf{X}$**.

If the dataset consists of $n$ points with $d$ dimensions, the size of the covariance matrix must be $d \times d$. Therefore, **the size of $\mathbf{C}$ does not depend on the number of data points**.

This makes sense: since the principal components are eigenvectors of $\mathbf{C}$, at most we can have as many components as the original dimensionality of the data.

In [ ]:
C = X.T @ X
C.shape

Didn't we forget something?

Before seeking the principal components, the data must be **centered**. This is because the principal components must not depend on the absolute position of the data in space, but rather by their distribution. Centering is as simple as:

In [ ]:
X = X - np.mean(X, axis=0)
C = X.T @ X

Let's now compute the first principal component of `X`:

In [ ]:
evals, evecs = np.linalg.eig(C)
idx = np.argsort(evals)[::-1]  # necessary because eig() does not return sorted eigenpairs
evals = evals[idx]
evecs = evecs[:, idx]

pc = evecs[:, 0]
pc

This is the principal axis of our data. Two trivial but nonetheless important observations arise:
- It has as many dimensions as the original data space.
- Its length and sign do not really matter: what matters is the **direction**.

Let's plot it:

In [ ]:
plt.figure(figsize=(5,2))
plt.scatter(X[:,0], X[:,1], s=5, color='red', zorder=3)
xx = np.linspace(-1, 1, 2)
plt.plot(xx*pc[0], xx*pc[1], color='green')
plt.grid(True)
plt.show()

Try to change the sign of `pc`, does it have any effect?

> **EXERCISE:** Use the power interation method to compute the principal component of the data we used so far:
>
> - Initialize with a random $\mathbf{v}_0$.
> - Iteratively compute:
> $$ \mathbf{v}_{t+1} = \frac{\mathbf{Cv}_{t}}{\|\mathbf{Cv}_{t}\|} $$

In [ ]:
# ✏️ your solution here

In [ ]:
# @title 👀 Solution

v = np.random.rand(2, 1)
for i in range(100):
  v = C @ v
  v /= np.sqrt(v.T @ v)

pc, v

## SVD

In fact, it turns out that principal components can be computed without constructing the covariance matrix at all!

Let's apply the singular value decomposition to the data matrix $\mathbf{X}$. We get the factorization:

$$ \mathbf{X} = \mathbf{USV}^\top$$

where $\mathbf{U}$ and $\mathbf{V}$ are orthogonal (left and right singular vectors) and $\mathbf{S}$ is diagonal (singular values).

> **EXERCISE:** _(pen and paper)_
> Consider the covariance matrix $\mathbf{X}^\top \mathbf{X}$. Substitute $\mathbf{X}$ with its SVD. What expression do you get?

If you did the derivations correctly, you should be convinced that we can equivalently compute the _right singular vectors_ of $\mathbf{X}$ instead of the eigenvectors of $\mathbf{C}$.

Let's verify it:

In [ ]:
U, s, Vt = np.linalg.svd(X, full_matrices=False)
V = Vt.T  # svd() returns the transpose of V

V[:,0], pc

Of course, the equivalence will always be up to sign.

> **EXERCISE:** We have seen that you can get at most as many principal components as there are dimensions in the data space. But can you get more principal components than there are _points_ in the dataset?
>
> _Hint:_ Think of what it means to have a few points in many dimensions.
>
> _Hint #2:_ Check what `full_matrices=False` means.

## Scikit-learn

Any ML library with basic functionalities can compute the PCA, and Scikit-learn is obviously one of those.

In [ ]:
from sklearn import decomposition

pca = decomposition.PCA(n_components=1)
_ = pca.fit(X)

pca.components_, pc

Feel free to use whatever approach you prefer. As always, make sure to check the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) of the libraries you use, not to miss any important details. For example, `sklearn`'s PCA also automatically centers the data, so we don't need to do it ourselves.

> **EXERCISE:** Generate a random dataset of 2D points, uniformly sampled from the interior of a 2D ellipse with different sample densities. Then:
> - Find its _two_ principal components and draw them as lines.
> - Do the same with a _circle_ instead of an ellipse.
>
> What conclusions can you draw from these tests?
>
> _Hint:_ to generate an ellipse, start from a circle and then rescale one of the two axes.

In [ ]:
density = [20, 100, 500, 1000]

# ✏️ your solution here

In [ ]:
# @title 👀 Solution

density = [20, 100, 500, 1000]

for scale in [0.5, 1]:  # ellipse and circle

  fig, ax = plt.subplots(1, len(density))
  fig.suptitle("circle" if scale == 1 else "ellipse")

  for i in range(len(density)):

    n = density[i]

    angles = np.random.uniform(low=0, high=2*np.pi, size=n)
    radii = np.sqrt(np.random.uniform(low=0, high=1, size=n))  # sqrt correction avoids higher density near the center

    x = radii * np.cos(angles) * scale
    y = radii * np.sin(angles)

    pca = decomposition.PCA(n_components=2)
    _ = pca.fit(np.hstack((x[:, None], y[:, None])))

    pc1 = pca.components_[:, 0]
    pc2 = pca.components_[:, 1] * scale

    ax[i].scatter(x, y, alpha=0.3, s=5)
    ax[i].plot([0, pc1[0]], [0, pc1[1]], color='red', zorder=3)
    ax[i].plot([0, pc2[0]], [0, pc2[1]], color='green', zorder=3)
    ax[i].set_title(f"n = {n}")
    ax[i].axis('equal')
    ax[i].axis('off')

## Dimensionality reduction

PCA provides a means to project the input data onto a lower-dimensional subspace of the data space. So far we have only looked at the principal axes -- but what about the projections themselves?

If we stack together the principal components in a matrix $\mathbf{W}$, we can obtain lower-dimensional representations $\mathbf{Z}$ via the orthogonal projection:

$$ \mathbf{Z} = \mathbf{W}^\top \mathbf{X} $$

In code:

In [ ]:
# reusing the 2D data from the beginning of the notebook.
# go re-run the first cells if you overwrote X.

pca = decomposition.PCA(n_components=1)
_ = pca.fit(X)

Z = X @ pca.components_.T
Z.shape

Scikit-learn performs the operation for us by calling:

In [ ]:
Z = pca.transform(X)
Z.shape

We get a new representation of our original dataset, with **fewer dimensions for each data point** (one instead of two, in this example). We just did dimensionality reduction, which was our original motivation to study PCA -- mission accomplished!🎯

As we can expect, this reduction is **lossy**: we can't reconstruct the original data exactly, because some details were lost in the projection.

Mathematically:

$$ \mathbf{X} \approx \mathbf{WZ} $$

Let's try it:

In [ ]:
X_recon = Z @ pca.components_

We get back to two dimensions. If we plot them, we see that they correspond to the original points _projected_ onto the principal axis:

In [ ]:
plt.figure(figsize=(5,2))
xx = np.linspace(-1, 1, 2)
plt.plot(xx*pca.components_[0,0], xx*pca.components_[0,1], color='green')
plt.scatter(X[:,0], X[:,1], s=5, color='red', zorder=2, alpha=0.2, label='original')
plt.scatter(X_recon[:,0], X_recon[:,1], edgecolor='orange', facecolors='white', zorder=3, label='reconstructed')
plt.grid(True)
plt.legend()
plt.show()

Intuitively, projecting onto the principal components allows us to **only keep the information that explains most of the variability in the data**, while discarding all the rest.

It has a compression effect, because for each data point we only store one number per principal component. If the data is structured, the compression can be significant without having to sacrifice too much information!

> **EXERCISE:** For the given set of 3D points given below, do the following:
> - Compute their projection onto the **first two** principal components.
> - How do you think they look like, if you plot the projections in 2D? Think about it, and then create the plot.
> - Re-do the same, but this time project onto the **first** principal component.

In [ ]:
n = 70
t = np.linspace(0, 6*np.pi, n)
x = t*np.cos(t)
y = 2*t*np.sin(t)
z = 0.1*t
X = np.stack((x, y, z)).T

# ✏️ your solution inbetween here

fig = make_subplots(rows=1, cols=3, specs=[[{'type': 'scene'}, {'type': 'xy'}, {'type': 'xy'}]])
fig.add_trace(go.Scatter3d(x=X[:,0], y=X[:,1], z=X[:,2], mode='markers', marker=dict(opacity=1, size=5, color=t, colorscale="Agsunset")), row=1, col=1)
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
# @title 👀 Solution

n = 70
t = np.linspace(0, 6*np.pi, n)
x = t*np.cos(t)
y = 2*t*np.sin(t)
z = 0.1*t
X = np.stack((x, y, z)).T

pca = decomposition.PCA(n_components=2)
_ = pca.fit(X)
Z2 = pca.transform(X)

pca = decomposition.PCA(n_components=1)
_ = pca.fit(X)
Z1 = pca.transform(X)

fig = make_subplots(rows=1, cols=3, specs=[[{'type': 'scene'}, {'type': 'xy'}, {'type': 'xy'}]])
fig.add_trace(go.Scatter3d(x=X[:,0], y=X[:,1], z=X[:,2], mode='markers', marker=dict(opacity=1, size=5, color=t, colorscale="Agsunset")), row=1, col=1)
fig.add_trace(go.Scatter(x=Z2[:,0], y=Z2[:,1], mode='markers', marker=dict(opacity=1, size=10, color=t, colorscale="Agsunset")), row=1, col=2)
fig.add_trace(go.Scatter(x=np.arange(n), y=Z1.reshape(-1), mode='markers', marker=dict(opacity=1, size=10, color=t, colorscale="Agsunset")), row=1, col=3)
fig.update_layout(showlegend=False)
fig.show()

# Learning a parameter space

Let's delve into one of the most fascinating aspects of PCA: its ability to **learn a parametric space**. This means that you can see the principal components as **parameters**. These parameters offer us the power to manipulate specific characteristics found in the original data space.

To bring this concept to life, we'll learn a set of parameters to control facial expressions within a 3D dataset! 😀😃😆

⚠️ This part requires using external data (find the links in the [course website](https://erodola.github.io/ML-s2-2024/)). Once you have downloaded the data, you can:
- Download the notebook itself and work locally, or...
- Mount your Google Drive, put the data there, and keep working from Colab.

To mount your drive, do this:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Data loading from Colab will probably be slow at first because it must copy all the shape files to the server. But it will be faster at subsequent runs.

In [ ]:
# @title Load shape data (change the paths if you work locally)

id = 'FaceTalk_170731_00024_TA'
n_verts = 5023

expressions = {
    'bareteeth': 122,
    'cheeks_in': 130,
    'eyebrow': 175,
    'high_smile': 140,
    'lips_back': 120,
    'lips_up': 136,
    'mouth_down': 176,
    'mouth_extreme': 28,
    'mouth_middle': 173,
    'mouth_open': 16,
    'mouth_side': 56,
    'mouth_up': 186
}

n_total_shapes = np.sum(list(v for _, v in expressions.items()))
X_ = np.empty((n_verts * 3, n_total_shapes), dtype=np.float32)
tris = np.array([])

j = 0
for k, v in expressions.items():
    for i in tqdm(range(1, v + 1)):
        fname = f"./drive/MyDrive/Colab Notebooks/ML/{id}/{k}/{k}.{i:06d}.ply"
        try:
            mesh = trimesh.load_mesh(fname)
            if tris.size == 0:
                tris = mesh.faces
            X_[:, j] = np.array(mesh.vertices).reshape(-1)
            j += 1
        except ValueError as e:
            print(f"Missing file: {e}")
X_ = X_[:, :j]  # in case a file was missing

n_shapes = X_.shape[1]

print('')
print('')
print(f"Number of shapes: {n_shapes}")
print(f"Number of vertices per shape: {n_verts}")

Before we forget (I did, when preparing this notebook!) and do anything else, let's **center the data**:

In [ ]:
X = np.empty((n_shapes, n_verts, 3))

for i in range(n_shapes):
  Y = X_[:,i].reshape(n_verts,3)
  X[i, :, :] = Y - np.mean(Y, axis=0)
X = X.reshape([n_shapes, 3*n_verts])

We have now created our data matrix, storing a shape per _row_. Each row contains all the (x, y, z) vertex coordinates of a shape, flattened into a 1D vector:

In [ ]:
X.shape

Let's look at a random shape from the dataset. We'll render it as a 2D image to keep things as readable as possible:

In [ ]:
idx = 609

# use this code to create a 3D mesh for plotting
mesh = trimesh.Trimesh(vertices=X[idx,:].reshape(n_verts, 3), faces=tris)

# use this code to plot a rendered 3D mesh
plt.figure(figsize=(5,2))
plt.imshow(np.rot90(render(mesh)), cmap='gray')
plt.axis('off')
plt.show()

Imagine having a parameter that lets you adjust how wide the mouth opens. That's exactly what we're going for!

But before we do that, we need to **center the data**. Again.

Why are we recentering? Because what we did before was to center each shape with respect to its (x, y, z) barycenter. Instead, PCA demands to center the **data matrix** such that the mean over its rows (each representing a data point) is zero!

In [ ]:
# center the flattened shape vectors
M = np.mean(X, axis=0)
X = X - M[None, :]

In other words, while in our early examples we were computing the mean of a set of 2D points and subtracted it from all points, now we are computing the **mean shape** and subtracting it from all the shapes. We can actually look at the mean shape:

In [ ]:
mesh = trimesh.Trimesh(vertices=M.reshape(n_verts, 3), faces=tris)

plt.figure(figsize=(5,2))
plt.imshow(np.rot90(render(mesh)), cmap='gray')
plt.axis('off')
plt.show()

> **EXERCISE:** What do you think will happen if you now plot again a random shape, _after_ the data has been centered? Think about it, and check your intuition by visualizing a random shape.

---

**A note on dimensionality**

You might have noticed that in our earlier toy examples we had $n$ data points in $d$ dimensions, with $n \gg d$.

Now it's quite the opposite: we are dealing with $n \ll d$ data points (around 1400 shapes, each with thousands of dimensions). Yet, since we are dealing with _structured_ data, we can rightfully hope that there is something to learn.

---

We are now ready to compute the principal components of `X`!

In [ ]:
n_pc = 4  # principal components

In [ ]:
pca = decomposition.PCA(n_components=n_pc)
_ = pca.fit(X)
V = pca.components_.T

# equivalent, but slower
# _, s, Vt = np.linalg.svd(X, full_matrices=False)
# V = Vt.T
# V = V[:, :n_pc]
# s = s[:n_pc]

Let's choose a shape and project it onto the principal components of the dataset:

In [ ]:
shape_idx = 0

Z = X[shape_idx] @ V

X[shape_idx].shape, Z.shape

Think about it: we took a data point that originally had 15,069 dimensions (5023 vertices times 3 vertex coordinates), and reduced it to mere **4 dimensions**! This seems like a very convenient encoding. But is it working any good? Let's try to reconstruct the shape, from `Z` back to `X`:

In [ ]:
X_recon = Z @ V.T
X_recon = X_recon + M[None, :]  # adding back the mean!

mesh = trimesh.Trimesh(vertices=(X[shape_idx] + M[None, :]).reshape(n_verts, 3), faces=tris)
mesh_recon = trimesh.Trimesh(vertices=X_recon.reshape(n_verts, 3), faces=tris)

fig, ax = plt.subplots(1, 2, figsize=(5, 2))

ax[0].imshow(np.rot90(render(mesh)), cmap='gray')
ax[0].axis('off')
ax[0].set_title('original')

ax[1].imshow(np.rot90(render(mesh_recon)), cmap='gray')
ax[1].axis('off')
ax[1].set_title('reconstructed');

Not bad, but if you look closely you'll see that the shape is not exactly the same. Some details are gone with the projection!

We'll now focus on something more interesting: how can we interpret the $\mathbf{Z}$ encoding?

## Interpreting $\mathbf{Z}$ as parameters 🎛️

The idea is that the 4 values encoded inside $\mathbf{Z}$ (the result of projecting the data $\mathbf{X}$ onto the principal components) can be interpreted as shape parameters.

Let's plot them:

In [ ]:
plt.figure(figsize=(4,2))
plt.bar(range(4), Z, zorder=2, width=0.5)
plt.grid('on')
plt.xlabel('Parameters')
plt.ylabel('Values')
plt.xticks(range(4), range(4))
plt.show()

You might be getting negative values; but as we know, the sign does not matter since these values multiply the principal components, which are defined up to sign.

Actually, it is very tempting to see these parameters as knobs that tune certain features in the data 🎚️🎛️. For example, why is parameter 2 so small? And why is parameter 0 so big in comparison? What are they representing?

Let's find out!

> **EXERCISE:** Introduce a scale factor $\alpha$ to selectively amplify one of the principal components, and verify what happens to the reconstructed shape. Test with a range of $\alpha$'s, and plot the resulting reconstructed shapes.

In [ ]:
shape_idx = 0  # shape
pc_idx = 0     # principal component

alphas = np.array([-6.5, -5, -2, 1, 2, 5, 6.5])

# ✏️ your solution here

In [ ]:
# @title 👀 Solution

shape_idx = 0  # shape
pc_idx = 0     # principal component

alphas = np.array([-6.5, -5, -2, 1, 2, 5, 6.5])

fig, ax = plt.subplots(1, 7, figsize=(10, 4))

for i, alpha in enumerate(alphas):

  Z = X[shape_idx] @ V
  Z[pc_idx] *= alpha

  X_recon = Z @ V.T + M[None, :]

  mesh = trimesh.Trimesh(vertices=X_recon.reshape(n_verts, 3), faces=tris)

  ax[i].imshow(np.rot90(render(mesh)), cmap='gray')
  ax[i].axis('off')

> **EXERCISE:** Re-run the previous experiment with another shape (e.g. `shape_idx=1181`). Is your intuition confirmed?

> **EXERCISE:** Test all the 4 components, including their combinations (e.g. scale all of them simultaneously, or only two of them, etc.). Are they all easy to interpret? _Do you need $\alpha$ with different magnitudes to observe an effect with all parameters?_

There is another way to analyze the learned principal components. Rather than choosing a specific shape and reconstructing it via a rescaled $\mathbf{Z}$, a better idea is to directly analyze the effect of the principal components on the **mean shape**.

The intuition is that we can modify the mean shape by adding individual components to it. Each component represents a specific direction of change, influencing certain shape features. By adjusting the scale of these added components, we can increase or decrease the variations captured by each one. This method is straightforward to implement — **simply add the component to the mean shape**.

But are all components equally important?

## Interpreting the singular values

(_Note: Before reading on, make sure you did the pen-and-paper exercise in the SVD section._)

Admittedly we have neglected the singular values so far, as we concentrated on the principal components themselves. However, we know from theory that the singular values of $\mathbf{X}$ (i.e. the square root of the eigenvalues of $\mathbf{C}$) are related to the **variability of the data** along each principal direction.

Let's plot the ones we have:

In [ ]:
plt.figure(figsize=(4,2))
plt.bar(range(4), pca.singular_values_, zorder=2, width=0.5, color='skyblue')
plt.grid('on')
plt.xlabel('Singular values')
plt.xticks(range(4), range(4))
plt.show()

This is sure suggesting that the first principal component explains a lot of the variability in the data! In fact, let's have a look at the first 200, rather than just the first four:

In [ ]:
pca_full = decomposition.PCA(n_components=200)
_ = pca_full.fit(X)

plt.figure(figsize=(5,2))
plt.plot(pca_full.singular_values_, zorder=2, color='skyblue', linewidth=4)
plt.grid('on')
plt.xlabel('Singular values')
plt.show()

They drop very quickly, meaning that indeed most of the information about the data is contained in the first few principal components! Probably 25-50 components are more than enough to parametrize the shape space of these facial expressions. We'll keep using four, just for ease of visualization.

In [ ]:
# recompute in case we overwrote something during our tests
pca = decomposition.PCA(n_components=n_pc)
_ = pca.fit(X)
V = pca.components_.T

When plotting deviations from the mean shape using principal components in PCA, we should then scale the principal component by its corresponding weight. This scaling ensures that modifications to the mean shape are realistic, consistent, and representative of actual variations observed in the dataset.

More precisely, for each principal component we can obtain its **variance** from the corresponding singular value $\sigma$ as:

$$ \sigma \mapsto \frac{\sigma^2}{n-1} $$

In fact, recall from the theory class that the variance is given by the _eigenvalues of the covariance matrix $\mathbf{X}^\top\mathbf{X}$_, which are the square of the singular values of the data matrix $\mathbf{X}$. We then divide by $n-1$, because the covariance matrix represents a raw sum of squares over $n$ data samples, rather than an average.

In [ ]:
var = (pca.singular_values_**2) / (n_shapes - 1)

# sklearn already does the calculation for us, compare:
var, pca.explained_variance_

Finally, let's plot each principal component at 2 standard deviations from the mean (each component using its own standard deviation value) to capture the variability across the majority of the data (95% in a normally distributed dataset):

In [ ]:
sigma = var**0.5

fig, ax = plt.subplots(2, n_pc, figsize=(10, 4))

for i in range(n_pc):

  A = M - 2*sigma[i]*V[:,i]
  mesh = trimesh.Trimesh(vertices=A.reshape(n_verts, 3), faces=tris)

  ax[0, i].imshow(np.rot90(render(mesh)), cmap='gray')
  ax[0, i].axis('off')
  ax[0, i].set_title(f"pc {i}")

  A = M + 2*sigma[i]*V[:,i]
  mesh = trimesh.Trimesh(vertices=A.reshape(n_verts, 3), faces=tris)

  ax[1, i].imshow(np.rot90(render(mesh)), cmap='gray')
  ax[1, i].axis('off')
  ax[1, i].set_title(f"pc {i}")

As a final note, Scikit-learn also computes the mean `M` for us, simplifying the overall pipeline to:

```python
pca = decomposition.PCA(n_components=n_pc)
pca.fit(X)
A_plus = pca.mean_ + 2*np.sqrt(pca.explained_variance_)*pca.components_
A_minus = pca.mean_ - 2*np.sqrt(pca.explained_variance_)*pca.components_
```

# Playground: The PCA Zoo 🦒

In this final part, you'll generate new animals weirder than the platypus!

Follow the same approach as we did for the 3D faces, but use the 3D animal data from the [SMAL dataset](https://smal.is.tue.mpg.de/). The data is available on the [course website](https://erodola.github.io/ML-s2-2024/).

If you mounted your Google Drive, you can use this code for loading:

In [ ]:
# @title Load shape data (change the paths if you work locally)

id = 'animals'

n_verts = 3889
n_shapes = 41

X_ = np.empty((n_verts * 3, n_shapes), dtype=np.float32)
tris = np.array([])

for i in tqdm(range(n_shapes)):
  fname = f"./drive/MyDrive/Colab Notebooks/ML/{id}/toy_{i}.ply"
  mesh = trimesh.load_mesh(fname)

  # rotate the mesh for better visualization
  R = trimesh.transformations.rotation_matrix(-np.pi/2, [1, 0, 0], point=None)
  mesh.apply_transform(R)
  R = trimesh.transformations.rotation_matrix(-np.pi/5, [0, 1, 0], point=None)
  mesh.apply_transform(R)

  if tris.size == 0:
    tris = mesh.faces
  X_[:, i] = np.array(mesh.vertices).reshape(-1)

print('')
print('')
print(f"Number of shapes: {n_shapes}")
print(f"Number of vertices per shape: {n_verts}")

I'll also **recenter the 3D shapes** for you, but you'll have to do all the rest by yourself!

In [ ]:
X = np.empty((n_shapes, n_verts, 3))

for i in range(n_shapes):
  Y = X_[:,i].reshape(n_verts,3)
  X[i, :, :] = Y - np.mean(Y, axis=0)
X = X.reshape([n_shapes, 3*n_verts])

X.shape

And here's the code for plotting a shape (notice the `height_pix=350` parameter, to increase the render resolution a bit):

In [ ]:
idx = 12

# use this code to create a 3D mesh for plotting
mesh = trimesh.Trimesh(vertices=X[idx,:].reshape(n_verts, 3), faces=tris)

# use this code to plot a rendered 3D mesh
plt.figure(figsize=(5,3))
plt.imshow(np.rot90(render(mesh, height_pix=350)), cmap='gray')
plt.axis('off')
plt.show()

On to you: try to learn some interesting parameters and use them to generate funny animals 🐒

> **EXERCISE:**
>
> - Train PCA on the SMAL dataset of 3D animals, and experiment with their principal components to control the generation of new animals.
> - How many components do we need to capture most of the variability of the data?
> - Try to **interpolate** the parameters between different shapes, can you get weird animal fusions?
> - Try to **extrapolate** the parameters by amplifying their values beyond boundaries: can you get exaggerated shape features?
> - Can you get a super fat lion? 🦁